In [108]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
import pickle
import os

In [6]:
# 웹 드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get("https://www.siksinhot.com/search?keywords=%EC%84%9C%EC%9A%B8")

In [ ]:
search_list = [
    "강동구","강서구","강남구","도봉구","양천구","마포구","서대문구","은평구","서초구","구로구","중구",
    "동대문구","중랑구","광진구","송파구","강북구","관악구"
]

In [ ]:
driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div").click()

In [ ]:
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys("강북구")
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys(Keys.RETURN)

In [ ]:
all_pages = []
for search in search_list:
    input_box = driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div")
    input_box.click()
    time.sleep(4)
    
    search_input = driver.find_element(By.CSS_SELECTOR,"#searchInput")
    search_input.send_keys(search)
    search_input.send_keys(Keys.RETURN)
    time.sleep(4)
    try:
        while True:
            more_box = driver.find_element(By.CSS_SELECTOR, "#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > a")

            more_box.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except:
        page_source = driver.page_source
        all_pages.append(page_source)
        pass 
    
    

In [ ]:
len(all_pages)

In [ ]:
for page in all_pages:
    soup = bs(page,"lxml")
    soup.select()

In [14]:
url = "https://www.siksinhot.com/search?keywords=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC"
r = requests.get(url)
soup = bs(r.text,"lxml")


In [15]:
main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")

In [7]:
page_list = []

for page in all_pages:
    soup = bs(page,"lxml")
    main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")
    for i in main:
        dict_list = {}
        title = i.select_one("h2").text
        location = i.select_one(".cate>a:first-child").text
        division = i.select_one(".cate>a:last-child").text
        view = i.select_one(".counting").text
        score = i.select_one(".score").text
        book_mark = i.select_one(".counting>span:nth-of-type(2)").text
        url = i.select_one("figure > a").get('href')
        address = i.select_one("figure > a > img").get('alt').split(",")[-1] if i.select_one("figure > a > img") is not None else "None"
        
    
        dict_list["제목"] = title 
        dict_list["지역"] = location
        dict_list["분류"] = division
        dict_list["조회수"] = view
        dict_list["평점"] = score
        dict_list["북마크"] = book_mark
        dict_list["url"] = url
        dict_list["주소"] = address
        
        page_list.append(dict_list)

NameError: name 'all_pages' is not defined

In [8]:
page_list

[]

In [ ]:
data = pd.DataFrame(page_list)

In [9]:
data.to_csv("식신.csv",encoding="utf-8",index=False)

NameError: name 'data' is not defined

In [26]:
data = pd.read_csv("식신.csv")

In [27]:
data[data["제목"].str.contains("꼼박사")]

,제목,지역,분류,조회수,평점,북마크,url,주소
1027,꼼박사 건대자양본점,건대/자양/화양,해물탕/해물요리,40892,3.0,2.0,https://www.siksinhot.com/P/364407,서울특별시 광진구 아차산로 354


In [28]:
len(data)

2028

In [13]:
detail_pages = [] 
for links in data["url"][1028:]:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
    driver.get(links)
    time.sleep(3)
    
    try:
        while True:
            more_box2 = driver.find_element(By.CSS_SELECTOR, "#storeView > div.container > div.store_review > a")

            more_box2.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except: 
        page_source2 = driver.page_source
        detail_pages.append(page_source2)
        driver.quit()
    

In [16]:
len(detail_pages)

1000

In [4]:
with open("식신디테일정보.pkl","rb") as read_file:
    main_pages = pickle.load(read_file)

In [29]:
len(main_pages)

2028

In [18]:
main_pages.extend(detail_pages)

In [9]:
main_pages[-1]

'<html><head>\n        <title>꼼박사 건대자양본점 - 서울 강북, 건대 자양 화양 | 맛집검색 식신</title>\n        <meta data-react-helmet="true" charset="utf-8"><meta data-react-helmet="true" http-equiv="Content-Type" content="text/html; charset=utf-8"><meta data-react-helmet="true" name="title" content="꼼박사 건대자양본점 - 서울 강북, 건대 자양 화양 | 맛집검색 식신"><meta data-react-helmet="true" name="description" content="꼼박사 건대자양본점입니다. 꼬들꼬들 맛있는 꼼장어입니다. 서울 강북, 건대 자양 화양에 위치하고 있습니다. 영업시간은 매일 18:00 ~ 02:00입니다. 대표메뉴는 꼼장어 양념소금(2인분), 아나고입니다."><meta data-react-helmet="true" name="keywords" content="맛집, 서울 강북, 건대 자양 화양맛집, 맛집, 맛집, 꼼장어 양념소금(2인분)맛집, 씨푸드맛집, 꼼박사 건대자양본점"><meta data-react-helmet="true" name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi"><meta data-react-helmet="true" name="robots" content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1 "><meta data-react-helmet="true" name="format-detection" content="telep

In [20]:
# 이거 진짜 신중하게 써라ㅠㅠㅠㅠㅠ덮어지뮤ㅠㅠㅠㅠㅠㅠ으어 내 이틀
with open("메인디테일정보.pkl","wb") as file:
    pickle.dump(main_pages,file)

In [21]:
len(main_pages)

2028

In [2]:
with open("메인디테일정보.pkl","rb") as read_file:
    main_pages = pickle.load(read_file)

In [3]:
len(main_pages)

2028

In [30]:
url = "https://www.siksinhot.com/search?keywords=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC"
r = requests.get(url)
soup = bs(r.text,"lxml")

In [75]:
for k in bs(main_pages[2],"lxml").select(".info_text"):
    print(k.text)

매장소개정말 맛있는 쌀국수 집으로 유명한 강동 호치민 입니다.  호치민은 쌀국수, 짜조, 비빔국수, 볶음밥을 맛있게 즐기실수 있습니다. 영업시간매일11:00 ~ 22:00OPEN메뉴쌀국수9,000 원볶음밥9,000 원짜조7,000 원쌀국수10,000 원짜조8,000 원볶음밥9,000 원비빔국수10,000 원해산물쌀국수11,000 원반미 샌드위치8,000 원주소서울특별시 강동구 천호대로 1089 복합시설동 지번서울특별시 강동구 천호동 447-17 강동 헤르셔 복합시설동 전화번호02-474-0780


In [103]:
with open("메인디테일정보.pkl","rb") as read_file:
    main_pages = pickle.load(read_file)

In [127]:
# 가게 이름을 담을 리스트
store_names = []
# 댓글 저장할 리스트
review_list = []

for page_review in main_pages[:5]:
    store_name = bs(page_review, "lxml").select_one(".store_info_title")
    if store_name:
        print(f"가게이름: {store_name.get_text(strip=True)}\n")
        
    items = bs(page_review, "lxml").select(".place_review_list > ul > li")

    for index, item in enumerate(items, start=1):
         # 아이디 추출
        name_element = item.select_one(".name")
        if name_element:
            # 모든 <span> 태그를 제거한 후 텍스트 추출
            print(name_element.get_text(strip=True))

        # 평점 추출 
        score_element = item.select_one(".name .score")
        if score_element:
            print(score_element.text)

        # 댓글내용 추출
        review_text_element = item.select_one(".review_text")
        if review_text_element:
            print(review_text_element.text)

        # 좋아요 추출
        like_count_element = item.select_one(".review_status > button.like-cnt")
        if like_count_element:
            print(like_count_element.text)
        # 대댓글 추출
        num_replies = item.select_one(".review_status > button.reply-cnt").text
        print()
        
        review = {
            "Store_Name": store_name.get_text(strip=True) if store_name else None,
            "Username": name_element if 'username' in locals() else None,
            "Score": score_element if 'score' in locals() else None,
            "Comment": review_text_element if 'comment_text' in locals() else None,
            "Like_Count": like_count_element if 'like_count' in locals() else None,
            "Reply_Count": num_replies
        }
        
        review_list.append(review)
    
# 리스트를 pickle 파일로 저장
with open('review_data.pkl', 'wb') as file:
    pickle.dump(review_list, file)
# 가게 이름 리스트를 확인
# print(store_names)        

가게이름: 동구밖오리구이

5306084
4
할머니 모시고 가족끼리 가서 한마리 반이랑 버섯추가했는데 양도 겁나 많고 맛잇엇어요!!!콩나물도 볶아주고 밑반찬도 다양하니 죠아꾸요!!!!!잔치국수도 존맛!!!!마무리로 호박식혜 주셔서 너무 달달하니 입가심 잘해써용💕먹느라 바빠서 사진이 별로 없네요 ㅎㅅㅎ 부들부들 부드러운 오리고기는 사랑입니댜...
좋아요 0

3450025
5
완~~~~전 맛나요!! 요즘 조류인플루엔자때문에 걱정되긴 하는데 그래도 푹익혀먹으면 되니가욨!
좋아요 0

연애세포133
3
또 가고 싶네요
좋아요 0

킴데렐라3
3
신선생 오리구이 진짜 훌륭했던 편이에요
좋아요 0

IM파써블3
3
신선생 오리구이  입맛에 맞았어요
좋아요 0

칼로리파괴자3
3
신선생 오리구이 진짜 훌륭하고말고요
좋아요 0

레인보우Chu3
3
여러모로 좋은 편이네요
좋아요 0

Seungjin Kim3.5
3.5
정말 직원분들 모두 하나같이 친절하시고 맛도 좋습니다. 식사 후 죽과 단호박 식혜는 정말 별미에요! 근데 죽 양이 생각보다 꽤 많아서(본죽 1인분 양 정도? ㅋ)... 2명이서 가신다면 공기밥은 굳이 안시켜도 될 것 같아요. (죽이 사람 수만큼 양이 조절되어 나오는지는 모르겠네요^^;) 정말 강추하는 맛집입니다!
좋아요 0

Gunwoo Park3.5
3.5
완전 친절하고 전부 맛있네요 ^^ 오리고기도 오리고기지만 된장국이랑 죽도 무지 맛있어요! 배달도된답니다
좋아요 0

가게이름: 양철북(강동구청점)

비타민젤리3
3
가볼만한 편이에요
좋아요 0

epi11223
3
양구이 아주 먹을만했던거 같습니당
좋아요 1

다시태어날꺼야3
3
환상적인 맛! 이힝
좋아요 0

Meeso Park3.5
3.5
여배우 박신혜양이 오픈한 양철북
좋아요 0

가게이름: 강동호치민

4715743
3
 점심에는 항상 호치민으로 발길이 향할 것 같다 ㅠㅠ
좋아요 0

슌3
3
 강동역과 붙어있는 새로운 건물 뒷쪽에 숨어있는 베트남 쌀국수 맛집!베트남 음식의 맛을 느낄 수 있는 호

In [128]:
file_path2 = 'C:/bigdata/self_practice'

for filename in os.listdir(file_path2):
    file_path2 = os.path.join(file_path2, filename)
    if os.path.isfile(file_path2):  # 파일인지 확인
        with open(file_path2, 'r', encoding='utf-8') as file:
            file_content = file.read()
            print(f"File: {filename}")
            print(file_content)
            print("-" * 50)  # 각 파일의 내용 구분을 위해 구분선 출력
    

In [122]:
file_path = 'C:/bigdata/self_practice/review_data.pkl'

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        file_review = file.read()
        print(file_review)
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 다시 확인해주세요.")
except UnicodeDecodeError:
    print("파일을 UTF-8로 디코딩할 수 없습니다. 다른 인코딩 방식을 시도해보세요.")

파일을 UTF-8로 디코딩할 수 없습니다. 다른 인코딩 방식을 시도해보세요.


In [7]:
main_pages[1]

'<html><head>\n        <title>양철북(강동구청점) - 서울 강남, 천호 강동 둔촌 | 맛집검색 식신</title>\n        <meta data-react-helmet="true" charset="utf-8"><meta data-react-helmet="true" http-equiv="Content-Type" content="text/html; charset=utf-8"><meta data-react-helmet="true" name="title" content="양철북(강동구청점) - 서울 강남, 천호 강동 둔촌 | 맛집검색 식신"><meta data-react-helmet="true" name="description" content="양철북(강동구청점)입니다. 양대창 구이전문점.입니다. 서울 강남, 천호 강동 둔촌에 위치하고 있습니다. 영업시간은 매일 11:30 ~ 22:30입니다. 대표메뉴는 특양구이, 한우대창입니다."><meta data-react-helmet="true" name="keywords" content="맛집, 서울 강남, 천호 강동 둔촌맛집, 맛집, 맛집, 특양구이맛집, 고기맛집, 양철북(강동구청점)"><meta data-react-helmet="true" name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi"><meta data-react-helmet="true" name="robots" content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1 "><meta data-react-helmet="true" name="format-detection" content="telephone=no"><meta data-